In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings

# Modelling
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
#from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [4]:
df = pd.read_csv('data\StudentsPerformance.csv')

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\91932\AppData\Local\Temp\ipykernel_21024\2277631988.py:1: SyntaxWarning: invalid escape sequence '\S'
  df = pd.read_csv('data\StudentsPerformance.csv')


In [5]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


Preparing X and Y variables

In [7]:
X = df.drop(columns = ['math score'], axis = 1)
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [16]:
y = df['math score']    # to be predicted
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math score, dtype: int64

In [9]:
# Unique categories for each categorical feature(column)
print('Gender : ', end = " ")
print(df['gender'].unique())

print('Race/Ethnicity : ', end = " ")
print(df['race/ethnicity'].unique())

print('Parental Levels of Education : ', end = " ")
print(df['parental level of education'].unique())

print('Lunch : ', end = " ")
print(df['lunch'].unique())

print('test preparation course : ', end = " ")
print(df['test preparation course'].unique())

Gender :  ['female' 'male']
Race/Ethnicity :  ['group B' 'group C' 'group A' 'group D' 'group E']
Parental Levels of Education :  ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Lunch :  ['standard' 'free/reduced']
test preparation course :  ['none' 'completed']


In [10]:
# Create a column transformer with 2 types of transformers (OneHotEncoding, StandardScaling)
numeric_features = X.select_dtypes(exclude = 'object').columns 
categorical_features = X.select_dtypes(include = 'object').columns

print(numeric_features)
print(categorical_features)

Index(['reading score', 'writing score'], dtype='object')
Index(['gender', 'race/ethnicity', 'parental level of education', 'lunch',
       'test preparation course'],
      dtype='object')


In [11]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
ohe_transformer = OneHotEncoder()

# Now, the ColumnTransformer will combine these 2 encoding processes together (pipeline)
preprocessor = ColumnTransformer([
    ("OneHotEncoder", ohe_transformer, categorical_features),    # first categories will be encoded
    ("StandardScaler", numeric_transformer, numeric_features)     # then numeric features will be scaled
])

In [12]:
X = preprocessor.fit_transform(X)
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.18, random_state = 42)
print(f'Training data shape : {X_train.shape}')
print(f'Validation data shape : {X_test.shape}')

Training data shape : (820, 19)
Validation data shape : (180, 19)


In [18]:
# Function to return all evaulation parameters for the model
def evaluate_model(actual_value, predicted_value):
    mae = mean_absolute_error(actual_value, predicted_value)
    mse = mean_squared_error(actual_value, predicted_value)
    rmse = np.sqrt(mean_squared_error(actual_value, predicted_value))
    r2_square = r2_score(actual_value, predicted_value)
    return mae, mse, rmse, r2_square

In [ ]:
models = {
    'Linear Regression' : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "K-Neighbors Regressor" : KNeighborsRegressor(),
    "Decision Tree Regressor" : DecisionTreeRegressor(),
    "Random Forest Regressor" : RandomForestRegressor(),
    "AdaBoost Regressor" : AdaBoostRegressor(),
    "XGBRegressor" : XGBRegressor()
}
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)                    # Training each model on training data
    
    y_train_pred = model.predict(X_train)          # Making predictions on training and testing sets
    y_test_pred = model.predict(X_test)
    
    model_train_mae, model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance on Training set : \n')
    print('RMS : ',model_train_rmse)
    print('MAE : ',model_train_mae)
    print('MSE : ',model_train_mse)
    
    print('Model performance on Test set : \n')
    print('RMS : ',model_test_rmse)
    print('MAE : ',model_test_mae)
    print('MSE : ',model_test_mse)
    
    r2_list.append(model_test_r2)

In [20]:
# Results 
pd.DataFrame(list(zip(model_list, r2_list)), columns = ['Model Name', 'R2 Score']).sort_values(by = ['R2 Score'], ascending = False)

,Model Name,R2 Score
2,Ridge,0.890482
0,Linear Regression,0.889628
5,Random Forest Regressor,0.869279
6,AdaBoost Regressor,0.857617
7,XGBRegressor,0.841783
1,Lasso,0.834275
3,K-Neighbors Regressor,0.791069
4,Decision Tree Regressor,0.744945
